In [0]:
# Databricks notebook source
# Requiere: yfinance instalado en el cluster
# Widgets
dbutils.widgets.text("catalog", "ptd_dev")
dbutils.widgets.text("schema_bronze", "bronze")
dbutils.widgets.text("start_date", "2015-01-01")
dbutils.widgets.text("end_date", "")
dbutils.widgets.text("extra_tickers", "SPY,^VIX")  # comma-separated
dbutils.widgets.text("vendor_map_table", "")       # opcional: ptd_dev.bronze.ticker_vendor_map

catalog         = dbutils.widgets.get("catalog")
schema_bronze   = dbutils.widgets.get("schema_bronze")
start_date      = dbutils.widgets.get("start_date")
end_date        = dbutils.widgets.get("end_date") or None
extra_tickers   = [t.strip().upper() for t in dbutils.widgets.get("extra_tickers").split(",") if t.strip()]
vendor_map_tbl  = dbutils.widgets.get("vendor_map_table").strip()

spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema_bronze}")

from pyspark.sql import functions as F, types as T
import pandas as pd
import time

try:
    import yfinance as yf
except Exception as e:
    raise RuntimeError("Instalá yfinance en el cluster (Libraries o %pip install yfinance).") from e

# 1) Lista de tickers desde el universo + extras
tickers_df = spark.sql(f"SELECT DISTINCT UPPER(ticker) AS ticker FROM {catalog}.bronze.universe_sp100_snapshot")
tickers = [r["ticker"] for r in tickers_df.collect()]
for t in extra_tickers:
    if t not in tickers:
        tickers.append(t)

# 2) Vendor map (opcional): si existe, aplicar (por ejemplo para BRK.B -> BRK-B)
vendor_map = {}
if vendor_map_tbl:
    exists = len([r for r in spark.sql(f"SHOW TABLES IN {catalog}.bronze").collect() if r['tableName']==vendor_map_tbl.split(".")[-1]])>0
    if exists:
        vm = spark.sql(f"SELECT UPPER(ticker) AS ticker, vendor_symbol FROM {vendor_map_tbl}")
        vendor_map = {r['ticker']: r['vendor_symbol'] for r in vm.collect()}

def to_yf(sym: str) -> str:
    return vendor_map.get(sym, sym if sym.startswith("^") else sym.replace(".", "-"))

# 3) Descargar OHLCV
rows = []
failures = []
for i, tk in enumerate(tickers, 1):
    yf_sym = to_yf(tk)
    try:
        hist = yf.download(yf_sym, start=start_date, end=end_date, progress=False, auto_adjust=False, interval="1d")
        if hist is None or hist.empty:
            failures.append((tk, "empty"))
            continue
        hist.reset_index(inplace=True)
        hist.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low",
                             "Close": "close", "Adj Close": "adj_close", "Volume": "volume"}, inplace=True)
        hist["ticker"] = tk
        hist["source"] = "yfinance"
        rows.append(hist[["ticker","date","open","high","low","close","adj_close","volume","source"]])
    except Exception as e:
        failures.append((tk, str(e)))
    # Tranquilizar rate limiting
    time.sleep(0.2)

if not rows:
    raise RuntimeError("No se descargaron precios. Revisá conexión o yfinance.")

pdf = pd.concat(rows, ignore_index=True)
# A Spark DF
sdf = spark.createDataFrame(pdf.assign(ingestion_ts=pd.Timestamp.utcnow()))

# 4) Crear tabla y MERGE (idempotente)
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.bronze.prices_raw (
  ticker STRING,
  date   DATE,
  open   DOUBLE,
  high   DOUBLE,
  low    DOUBLE,
  close  DOUBLE,
  adj_close DOUBLE,
  volume   DOUBLE,
  source   STRING,
  ingestion_ts TIMESTAMP
)
USING DELTA
PARTITIONED BY (date)
""")

sdf.createOrReplaceTempView("tmp_prices_in")
spark.sql(f"""
MERGE INTO {catalog}.bronze.prices_raw AS t
USING (
  SELECT DISTINCT ticker, CAST(date AS DATE) AS date, open, high, low, close, adj_close, volume, source, ingestion_ts
  FROM tmp_prices_in
) s
ON t.ticker = s.ticker AND t.date = s.date
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")

print(f"Descargas fallidas: {len(failures)}")
if failures:
    display(spark.createDataFrame(failures, schema="ticker string, error string"))